In [1]:
from datasets import load_dataset

dataset = load_dataset('json', data_files='../../data/daccord_yifei_v2_leo_labelMerged.json', split='train')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /mnt/nlp4sd/hugging_face_cache/ysong/datasets/json/default-0540f20d53255dec/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51. Subsequent calls will reuse this data.


In [2]:
len(dataset)

113

In [3]:
dataset[0]

{'id': 'f4acjs9',
 'body_cleaned': 'as a federal leo , the very idea of confiscating guns is laughable . i swore an oath to the constitution , not to beto or any other politician .',
 'User label': 'support',
 'author': 'MDeXY',
 'subreddit': 'progun',
 'predicted_community': 0,
 'score': 454,
 'created_utc': 1571492410}

In [4]:
def convert_into_prompt_template(user_message, train=True, label_text=""):
    if train:
        text = f"<s>[INST] Sentence: {user_message} \nSentiment Polarity: [/INST] {label_text} </s>"
    else:
        text = f"<s>[INST] Sentence: {user_message} \nSentiment Polarity: [/INST] "

    return text

In [5]:
print(convert_into_prompt_template(dataset[0]['body_cleaned'], True, dataset[0]['User label']))

<s>[INST] Sentence: as a federal leo , the very idea of confiscating guns is laughable . i swore an oath to the constitution , not to beto or any other politician . 
Sentiment Polarity: [/INST] support </s>


In [6]:
def map_dataset(dataset, train=True):
    # Convert the dataset into the format required by the model
    def convert(sentence, label_text):
        converted_inputs = convert_into_prompt_template(sentence, train, label_text)
        return {'text': converted_inputs, 'label_text': label_text}

    return dataset.map(convert, input_columns=['body_cleaned', 'User label'], batched=False, remove_columns=['id', 'body_cleaned', 'User label', 'author', 'subreddit', 'predicted_community', 'score', 'created_utc'])

In [7]:
new_dataset = map_dataset(dataset, train=True)
new_validation_dataset = map_dataset(dataset, train=False)

Map:   0%|          | 0/113 [00:00<?, ? examples/s]

Map:   0%|          | 0/113 [00:00<?, ? examples/s]

In [8]:
new_dataset[0]

{'text': '<s>[INST] Sentence: as a federal leo , the very idea of confiscating guns is laughable . i swore an oath to the constitution , not to beto or any other politician . \nSentiment Polarity: [/INST] support </s>',
 'label_text': 'support'}

In [9]:
new_validation_dataset[0]

{'text': '<s>[INST] Sentence: as a federal leo , the very idea of confiscating guns is laughable . i swore an oath to the constitution , not to beto or any other politician . \nSentiment Polarity: [/INST] ',
 'label_text': 'support'}

In [10]:
from datasets import DatasetDict

dataset_to_upload = DatasetDict({
    'train': new_dataset,
    'validation': new_validation_dataset
})

In [11]:
dataset_to_upload

DatasetDict({
    train: Dataset({
        features: ['text', 'label_text'],
        num_rows: 113
    })
    validation: Dataset({
        features: ['text', 'label_text'],
        num_rows: 113
    })
})

In [12]:
dataset_to_upload.push_to_hub("OneFly7/llama2-politosphere-fine-tuning-supp-anti-others")

Pushing split train to the Hub.


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

Pushing split validation to the Hub.


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]